# 1. Import and Install Dependencies

In [84]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [85]:
mp_pose = mp.solutions.pose # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [86]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [87]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [88]:
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )

In [89]:
def extract_keypoints(results):
    target_indedices=[11,12,13,14,15,16,25,26,27,28,29,30,31,32]
    pose = np.array([[results.pose_landmarks.landmark[index].x, results.pose_landmarks.landmark[index].y, results.pose_landmarks.landmark[index].z, results.pose_landmarks.landmark[index].visibility] for index in target_indedices]).flatten() if results.pose_landmarks else np.zeros(len(target_indedices)*4)
    return pose

# 4. Setup Folders for Collection

In [161]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['left_shot', 'right_shot', 'head'])

# Thirty videos worth of data
no_sequences = 15

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 1

In [162]:
#create directory
# import shutil
# for action in actions:
        #shutil.rmtree(os.path.join(DATA_PATH, action));
        #os.makedirs(os.path.join(DATA_PATH, action),exist_ok=True)

In [168]:
print(max([int(f) for f in os.listdir(os.path.join(DATA_PATH, action)) if not f.startswith('.')]))


75


In [169]:
start_folder=max([int(f) for f in os.listdir(os.path.join(DATA_PATH, action)) if not f.startswith('.')])+1
print(start_folder)

76


# 5. Collect Keypoint Values for Training and Testing

In [171]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        start_folder=max([int(f) for f in os.listdir(os.path.join(DATA_PATH, action)) if not f.startswith('.')])+1
        for sequence in range(start_folder, start_folder+no_sequences):
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, pose)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(4000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [172]:
cap.release()
cv2.destroyAllWindows()

In [173]:
for action in actions:
    points=np.load(os.path.join(DATA_PATH, action,'1','0.npy'));
    print(len(points))
    break

56


# 6. Preprocess Data and Create Labels and Features

In [224]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [225]:
label_map = {label:num for num, label in enumerate(actions)}

In [226]:
label_map

{'left_shot': 0, 'right_shot': 1, 'head': 2}

In [227]:
sequence_length=15

In [228]:
sequences, labels = [], []
for action in actions:
    file_list = [file for file in os.listdir(os.path.join(DATA_PATH, action)) if file != '.DS_Store']
    file_array = np.array(file_list).astype(int)
    for sequence in file_array:
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [229]:
np.array(sequences).shape

(270, 15, 56)

In [230]:
np.array(labels).shape

(270,)

In [231]:
X = np.array(sequences)

In [232]:
X.shape

(270, 15, 56)

In [233]:
y = to_categorical(labels).astype(int)

In [234]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [235]:
y_test.shape

(14, 3)

# 7. Build and Train LSTM Neural Network

In [236]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [237]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [238]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(np.array(sequences).shape[1],np.array(sequences).shape[2])))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [239]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [240]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
8/8 [==============================] - 4s 17ms/step - loss: 1.1007 - categorical_accuracy: 0.3164
Epoch 2/2000
8/8 [==============================] - 0s 12ms/step - loss: 1.0984 - categorical_accuracy: 0.3086
Epoch 3/2000
8/8 [==============================] - 0s 12ms/step - loss: 1.0952 - categorical_accuracy: 0.4258
Epoch 4/2000
8/8 [==============================] - 0s 12ms/step - loss: 1.0866 - categorical_accuracy: 0.4297
Epoch 5/2000
8/8 [==============================] - 0s 12ms/step - loss: 1.0714 - categorical_accuracy: 0.4062
Epoch 6/2000
8/8 [==============================] - 0s 12ms/step - loss: 0.9842 - categorical_accuracy: 0.4883
Epoch 7/2000
8/8 [==============================] - 0s 12ms/step - loss: 1.0583 - categorical_accuracy: 0.5000
Epoch 8/2000
8/8 [==============================] - 0s 12ms/step - loss: 1.0120 - categorical_accuracy: 0.3867
Epoch 9/2000
8/8 [==============================] - 0s 11ms/step - loss: 0.9781 - categorical_accuracy: 0.6797
E

8/8 [==============================] - 0s 12ms/step - loss: 0.4377 - categorical_accuracy: 0.8320
Epoch 75/2000
8/8 [==============================] - 0s 11ms/step - loss: 0.4481 - categorical_accuracy: 0.8125
Epoch 76/2000
8/8 [==============================] - 0s 12ms/step - loss: 0.3801 - categorical_accuracy: 0.8438
Epoch 77/2000
8/8 [==============================] - 0s 12ms/step - loss: 0.3342 - categorical_accuracy: 0.8672
Epoch 78/2000
8/8 [==============================] - 0s 12ms/step - loss: 0.3256 - categorical_accuracy: 0.8906
Epoch 79/2000
8/8 [==============================] - 0s 11ms/step - loss: 0.2866 - categorical_accuracy: 0.8984
Epoch 80/2000
8/8 [==============================] - 0s 12ms/step - loss: 0.3287 - categorical_accuracy: 0.8867
Epoch 81/2000
8/8 [==============================] - 0s 13ms/step - loss: 0.3503 - categorical_accuracy: 0.8789
Epoch 82/2000
8/8 [==============================] - 0s 13ms/step - loss: 0.2974 - categorical_accuracy: 0.8945
Epoch 

8/8 [==============================] - 0s 12ms/step - loss: 0.2474 - categorical_accuracy: 0.9062
Epoch 147/2000
8/8 [==============================] - 0s 12ms/step - loss: 0.2497 - categorical_accuracy: 0.8984
Epoch 148/2000
8/8 [==============================] - 0s 11ms/step - loss: 0.2272 - categorical_accuracy: 0.9219
Epoch 149/2000
8/8 [==============================] - 0s 11ms/step - loss: 0.2576 - categorical_accuracy: 0.8984
Epoch 150/2000
8/8 [==============================] - 0s 12ms/step - loss: 0.2031 - categorical_accuracy: 0.9297
Epoch 151/2000
8/8 [==============================] - 0s 11ms/step - loss: 0.1793 - categorical_accuracy: 0.9453
Epoch 152/2000
8/8 [==============================] - 0s 11ms/step - loss: 0.1613 - categorical_accuracy: 0.9492
Epoch 153/2000
8/8 [==============================] - 0s 11ms/step - loss: 0.1820 - categorical_accuracy: 0.9375
Epoch 154/2000
8/8 [==============================] - 0s 11ms/step - loss: 0.3229 - categorical_accuracy: 0.886

8/8 [==============================] - 0s 12ms/step - loss: 0.3211 - categorical_accuracy: 0.8789
Epoch 219/2000
8/8 [==============================] - 0s 11ms/step - loss: 0.2970 - categorical_accuracy: 0.8828
Epoch 220/2000
8/8 [==============================] - 0s 11ms/step - loss: 0.2514 - categorical_accuracy: 0.9219
Epoch 221/2000
8/8 [==============================] - 0s 11ms/step - loss: 0.2344 - categorical_accuracy: 0.9258
Epoch 222/2000
8/8 [==============================] - 0s 12ms/step - loss: 0.2335 - categorical_accuracy: 0.9219
Epoch 223/2000
8/8 [==============================] - 0s 12ms/step - loss: 0.2325 - categorical_accuracy: 0.9219
Epoch 224/2000
8/8 [==============================] - 0s 12ms/step - loss: 0.2174 - categorical_accuracy: 0.9219
Epoch 225/2000
8/8 [==============================] - 0s 11ms/step - loss: 0.2151 - categorical_accuracy: 0.9219
Epoch 226/2000
8/8 [==============================] - 0s 11ms/step - loss: 0.2676 - categorical_accuracy: 0.921

8/8 [==============================] - 0s 12ms/step - loss: 0.2237 - categorical_accuracy: 0.9258
Epoch 291/2000
8/8 [==============================] - 0s 12ms/step - loss: 0.2360 - categorical_accuracy: 0.9258
Epoch 292/2000
8/8 [==============================] - 0s 11ms/step - loss: 0.2542 - categorical_accuracy: 0.9023
Epoch 293/2000
8/8 [==============================] - 0s 11ms/step - loss: 0.2336 - categorical_accuracy: 0.9180
Epoch 294/2000
8/8 [==============================] - 0s 12ms/step - loss: 0.1782 - categorical_accuracy: 0.9453
Epoch 295/2000
8/8 [==============================] - 0s 11ms/step - loss: 0.1610 - categorical_accuracy: 0.9531
Epoch 296/2000
8/8 [==============================] - 0s 12ms/step - loss: 0.1454 - categorical_accuracy: 0.9531
Epoch 297/2000
8/8 [==============================] - 0s 11ms/step - loss: 0.1463 - categorical_accuracy: 0.9492
Epoch 298/2000
8/8 [==============================] - 0s 11ms/step - loss: 0.1809 - categorical_accuracy: 0.929

...

8/8 [==============================] - 0s 15ms/step - loss: 0.1008 - categorical_accuracy: 0.9688
Epoch 1658/2000
8/8 [==============================] - 0s 15ms/step - loss: 0.1008 - categorical_accuracy: 0.9688
Epoch 1659/2000
8/8 [==============================] - 0s 14ms/step - loss: 0.1066 - categorical_accuracy: 0.9648
Epoch 1660/2000
8/8 [==============================] - 0s 14ms/step - loss: 0.1072 - categorical_accuracy: 0.9648
Epoch 1661/2000
8/8 [==============================] - 0s 14ms/step - loss: 0.1370 - categorical_accuracy: 0.9453
Epoch 1662/2000
8/8 [==============================] - 0s 14ms/step - loss: 0.1040 - categorical_accuracy: 0.9688
Epoch 1663/2000
8/8 [==============================] - 0s 14ms/step - loss: 0.0975 - categorical_accuracy: 0.9688
Epoch 1664/2000
8/8 [==============================] - 0s 14ms/step - loss: 0.0916 - categorical_accuracy: 0.9727
Epoch 1665/2000
8/8 [==============================] - 0s 15ms/step - loss: 0.0994 - categorical_accurac

8/8 [==============================] - 0s 15ms/step - loss: 0.1218 - categorical_accuracy: 0.9531
Epoch 1730/2000
8/8 [==============================] - 0s 15ms/step - loss: 0.1131 - categorical_accuracy: 0.9609
Epoch 1731/2000
8/8 [==============================] - 0s 15ms/step - loss: 0.0947 - categorical_accuracy: 0.9688
Epoch 1732/2000
8/8 [==============================] - 0s 15ms/step - loss: 0.0848 - categorical_accuracy: 0.9727
Epoch 1733/2000
8/8 [==============================] - 0s 15ms/step - loss: 0.0862 - categorical_accuracy: 0.9688
Epoch 1734/2000
8/8 [==============================] - 0s 15ms/step - loss: 0.0886 - categorical_accuracy: 0.9688
Epoch 1735/2000
8/8 [==============================] - 0s 16ms/step - loss: 0.0852 - categorical_accuracy: 0.9727
Epoch 1736/2000
8/8 [==============================] - 0s 15ms/step - loss: 0.0809 - categorical_accuracy: 0.9688
Epoch 1737/2000
8/8 [==============================] - 0s 15ms/step - loss: 0.1001 - categorical_accurac

8/8 [==============================] - 0s 18ms/step - loss: 0.0746 - categorical_accuracy: 0.9727
Epoch 1802/2000
8/8 [==============================] - 0s 17ms/step - loss: 0.0813 - categorical_accuracy: 0.9766
Epoch 1803/2000
8/8 [==============================] - 0s 17ms/step - loss: 0.0723 - categorical_accuracy: 0.9766
Epoch 1804/2000
8/8 [==============================] - 0s 21ms/step - loss: 0.0890 - categorical_accuracy: 0.9727
Epoch 1805/2000
8/8 [==============================] - 0s 18ms/step - loss: 0.0880 - categorical_accuracy: 0.9648
Epoch 1806/2000
8/8 [==============================] - 0s 17ms/step - loss: 0.0831 - categorical_accuracy: 0.9688
Epoch 1807/2000
8/8 [==============================] - 0s 17ms/step - loss: 0.0733 - categorical_accuracy: 0.9727
Epoch 1808/2000
8/8 [==============================] - 0s 17ms/step - loss: 0.1028 - categorical_accuracy: 0.9648
Epoch 1809/2000
8/8 [==============================] - 0s 16ms/step - loss: 0.0992 - categorical_accurac

8/8 [==============================] - 0s 19ms/step - loss: 0.0729 - categorical_accuracy: 0.9727
Epoch 1874/2000
8/8 [==============================] - 0s 17ms/step - loss: 0.0983 - categorical_accuracy: 0.9531
Epoch 1875/2000
8/8 [==============================] - 0s 18ms/step - loss: 0.1625 - categorical_accuracy: 0.9414
Epoch 1876/2000
8/8 [==============================] - 0s 19ms/step - loss: 0.3100 - categorical_accuracy: 0.9258
Epoch 1877/2000
8/8 [==============================] - 0s 18ms/step - loss: 0.4983 - categorical_accuracy: 0.8555
Epoch 1878/2000
8/8 [==============================] - 0s 17ms/step - loss: 0.3030 - categorical_accuracy: 0.9023
Epoch 1879/2000
8/8 [==============================] - 0s 17ms/step - loss: 0.1747 - categorical_accuracy: 0.9258
Epoch 1880/2000
8/8 [==============================] - 0s 18ms/step - loss: 0.1629 - categorical_accuracy: 0.9570
Epoch 1881/2000
8/8 [==============================] - 0s 18ms/step - loss: 0.1278 - categorical_accurac

8/8 [==============================] - 0s 19ms/step - loss: 0.1248 - categorical_accuracy: 0.9609
Epoch 1946/2000
8/8 [==============================] - 0s 18ms/step - loss: 0.1104 - categorical_accuracy: 0.9648
Epoch 1947/2000
8/8 [==============================] - 0s 18ms/step - loss: 0.1148 - categorical_accuracy: 0.9609
Epoch 1948/2000
8/8 [==============================] - 0s 18ms/step - loss: 0.0981 - categorical_accuracy: 0.9688
Epoch 1949/2000
8/8 [==============================] - 0s 18ms/step - loss: 0.0877 - categorical_accuracy: 0.9688
Epoch 1950/2000
8/8 [==============================] - 0s 18ms/step - loss: 0.0805 - categorical_accuracy: 0.9727
Epoch 1951/2000
8/8 [==============================] - 0s 18ms/step - loss: 0.0813 - categorical_accuracy: 0.9688
Epoch 1952/2000
8/8 [==============================] - 0s 18ms/step - loss: 0.0803 - categorical_accuracy: 0.9727
Epoch 1953/2000
8/8 [==============================] - 0s 18ms/step - loss: 0.0777 - categorical_accurac

In [241]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_18 (LSTM)              (None, 15, 64)            30976     
                                                                 
 lstm_19 (LSTM)              (None, 15, 128)           98816     
                                                                 
 lstm_20 (LSTM)              (None, 64)                49408     
                                                                 
 dense_18 (Dense)            (None, 64)                4160      
                                                                 
 dense_19 (Dense)            (None, 32)                2080      
                                                                 
 dense_20 (Dense)            (None, 3)                 99        
                                                                 
Total params: 185539 (724.76 KB)
Trainable params: 185

# 8. Make Predictions

In [242]:
res = model.predict(X_test)

1/1 [==============================] - 0s 451ms/step


In [243]:
actions[np.argmax(res[4])]

'left_shot'

In [244]:
actions[np.argmax(y_test[4])]

'left_shot'

# 9. Save Weights

In [198]:
model.save('action.h5')

/Users/alphabravo/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [207]:
model.load_weights('action.h5')

# 10. Evaluation using Confusion Matrix and Accuracy

In [245]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [246]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 19ms/step


In [247]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [248]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[ 9,  0],
        [ 0,  5]],

       [[11,  0],
        [ 1,  2]],

       [[ 7,  1],
        [ 0,  6]]])

In [249]:
accuracy_score(ytrue, yhat)

0.9285714285714286

# 11. Test in Real Time

In [250]:
from scipy import stats

In [251]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [252]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

TypeError: only size-1 arrays can be converted to Python scalars

<Figure size 1800x1800 with 0 Axes>

In [253]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.6

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, pose)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-sequence_length:]
        
        if len(sequence) == sequence_length:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 19ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [=====================

1/1 [==============================] - 0s 16ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/s

1/1 [==============================] - 0s 16ms/step
head
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
head
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
head
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
head
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
head
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
head
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
head
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
head
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
head
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
right_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
head
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
head
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
head
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
head
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
head
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
head
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
head
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [====================

1/1 [==============================] - 0s 17ms/step
head
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
head
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
head
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
head
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
head
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
head
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
head
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
head
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
right_shot
<class 'mediapipe.python.

1/1 [==============================] - 0s 16ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/s

1/1 [==============================] - 0s 16ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 15ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 17ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/step
left_shot
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 16ms/s

KeyboardInterrupt: 